# Parse example sentences
Prepare example sentence data set for merging with compound flashcards. Documentation about index entries: http://www.edrdg.org/wiki/index.php/Sentence-Dictionary_Linking.

Index entry format quick reference: `headword(reading)[sense number]{inflected form}`.

In [1]:
import pandas as pd

## Load source data

In [2]:
df = pd.read_csv('./data/example_sentences.csv', delimiter='\t',
                 names=['jpn_seq_no', 'eng_seq_no', 'jpn_sentence', 'eng_sentence', 'xref'])

In [3]:
df.sample(frac=0.0001)

,jpn_seq_no,eng_seq_no,jpn_sentence,eng_sentence,xref
49837,124716,279278,電話でメモが取れますか。,Can you take notes over the phone?,電話 で メモ が 取れる{取れます} か
68185,153954,260589,私は彼の健康が心配だった。,I was worried about his health.,私(わたし)[01] は|1 彼(かれ)[01]{彼の} 健康 が 心配 だ{だった}
3247,77239,326481,六月は来る日も来る日も雨が降る。,"In June, it rains day after day.",六月 は|1 来る日も来る日も 雨 が 降る(ふる)
51284,126185,277807,朝食を食べたので急いで学校に行った。,"Having finished breakfast, I hurried to school.",朝食 を 食べる{食べた} ので 急ぐ{急いで} 学校 に 行く[01]{行った}
136101,224393,61728,ここに鈍痛があります。,I have a dull ache here.,此処[01]{ここ} に 鈍痛~ が[01] 有る{あります}
78494,164353,250156,私に対する彼の口のきき方が気に入りません。,I don't like the way he speaks to me.,私(わたし)[01] に対する 彼(かれ) の 口の利き方{口のきき方} が 気に入る{気に...
51424,126331,277660,注文していない品物を受け取りました。,I received an item that I did not order.,注文 為る(する){していない} 品物 を 受け取る{受け取りました}
10561,84696,319018,父はちょうど帰ってきたところだ。,My father has just come home.,父 は|1 丁度{ちょうど} 帰る[01]{帰って} 来る(くる)[01]{きた} 所(とこ...
32447,106774,296912,彼は昨日東京へ行きました。,He went to Tokyo yesterday.,彼(かれ)[01] は|1 昨日 東京 へ 行く[01]{行きました}
117501,205654,42899,それには真実のかけらもない。,There is not a ray of truth in it.,其れ[01]{それ} には 真実 の 欠片{かけらもない}~


## Find and expand lemmas in index string

In [4]:
def find_lemmas(index_string):
    index_list = index_string.split(' ')
    lemma_list = [''] * len(index_list)
    
    for i, index in enumerate(index_list):
        lemma = index.split('|')[0].split('(')[0].split('[')[0].split('{')[0].replace('~', '')
        if '[' in index:
            sense = int(index[(index.find('[') + 1):index.find(']')])
        else:
            sense = None
        if '(' in index:
            reading = index[(index.find('(') + 1):index.find(')')]
        else:
            reading = None
        if '~' in index:
            checked = True
        else:
            checked = False
        lemma_list[i] = [lemma, reading, sense, checked]
        
    return lemma_list

In [5]:
df['lemmas'] = df['xref'].apply(find_lemmas)

In [6]:
df = df.explode('lemmas')

## Create separate columns for lemma, sense and checked flag

In [7]:
df['lemma'] = df['lemmas'].apply(lambda x: x[0])
df['reading'] = df['lemmas'].apply(lambda x: x[1])
df['sense'] = df['lemmas'].apply(lambda x: x[2])
df['checked'] = df['lemmas'].apply(lambda x: x[3])

## Export the processed data set

In [8]:
df = df[['jpn_seq_no', 'eng_seq_no', 'jpn_sentence', 'eng_sentence',
         'lemma', 'reading', 'sense', 'checked']].reset_index(drop=True)

In [9]:
df.sample(frac=0.00001)

,jpn_seq_no,eng_seq_no,jpn_sentence,eng_sentence,lemma,reading,sense,checked
527665,152301,262249,私は例えばイタリアやスペインなど外国へ行きたい。,"I want to go abroad, for instance, to Italy an...",例えば,None,NaN,False
954999,209222,1749362,その少女はいつも幸せそうに見えたが、決して幸せではなかった。,"The girl always looked happy, but she wasn't a...",は,None,NaN,False
11521,75077,328642,あ、ちょっとタンマ。クツのヒモがほどけちゃった。,"Ah, hold a mo. My shoelace's come undone.",たんま,None,NaN,True
854798,195771,32946,マイクはここ一年間失業中である。,Mike has been out of work for a year.,中,ちゅう,6.0,False
1023663,218008,55317,これら２本の線は直角に交差している。,These two lines cut across each other at right...,直角,None,NaN,False
53171,79808,323909,問題は彼女が信用できるかどうかということである。,The question is whether she can be trusted.,は,None,NaN,False
533367,153042,261503,私は普通８時に起きます。,I usually get up at eight.,普通,None,NaN,False
662964,168726,245765,子供たちは自分たち自身の遊びを考え出す傾向があった。,Children were apt to invent their own games.,が,None,1.0,False
255653,106190,297496,彼は私にそうするように命じた。,He commanded me to do it.,に,None,NaN,False
1072448,224180,61516,こちらには大きなハスの葉があります。,We have very big lotus leaves.,此方,こちら,NaN,False


In [10]:
df.to_parquet('./data/example_sentences_processed.parquet')